# Advanced Terraform Concepts

IMPORTANT: Any resources you provision on Azure will accrue a cost. With an Azure Free Tier account you are allowed a certain capacity free for each resource, which you can learn more about here: [Azure Free Tier account services](https://azure.microsoft.com/en-gb/free/search/?ef_id=_k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&OCID=AIDcmmiouhop3i_SEM__k_CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE_k_&gad_source=1&gclid=CjwKCAjwuJ2xBhA3EiwAMVjkVI3txRJhRTDRD6Y0RujfvSEgAdBRc1ANmQykYFPVSAw7Mh2H8BcnNRoCkjMQAvD_BwE).
Once you run out of credits on your free tier subscription you will be charged, so make sure to decomission any resources you create on this account once you are finished with them.

## Terraform State Management 

> *Terraform state* is a critical component of Terraform's workflow. It represents the current state of your infrastructure and serves as the source of truth for Terraform. The state file is a record of the relationships and properties of the resources defined in your Terraform configuration. It contains essential information such as resource IDs, IP addresses, and other attributes.

Terraform state is essential because it:

- **Tracks Resource State**: Terraform uses it to compare the desired state with the current state of resources, enabling precise updates
- **Manages Resource Dependencies**: It helps Terraform provision resources in the correct order and resolves dependencies accurately
- **Enables State Locking**: State locking prevents concurrent updates, enhancing stability and data integrity
- **Stores Critical Data**: It persists valuable infrastructure data for monitoring, troubleshooting, and auditing purposes

### Local vs Remote State File

By default, Terraform stores state information locally in a file named `terraform.tfstate`. The state file is created in the working directory where you run Terraform commands, unless you specify a different path or use a remote state backend.

It is a `JSON` or binary file that stores details about the resources defined in your configuration, such as their attributes and dependencies. Terraform uses this information to plan and apply changes to your infrastructure. You can check out the contents of this file for yourself after running `terraform apply` in your project directory.

This approach works well for small-scale projects and individual use cases. However, local state has limitations when it comes to team collaboration and larger projects. To address the limitations of local state, Terraform allows you to store state remotely using various backends. Remote state offers several advantages:

- **Collaboration**: Remote state enables collaboration among team members. Multiple users can work on the same infrastructure project without conflicts.

- **Concurrency Control**: Remote state backends often provide locking mechanisms to prevent concurrent modifications, ensuring data integrity

- **Security**: Sensitive data can be better protected in remote state backends. Access controls and encryption are typically available, reducing the risk of data exposure.

- **Scaling**: Remote state works well for larger projects and complex infrastructures where multiple environments (e.g., development, staging, production) need to be managed

Popular choices for remote state backends include:

- **Amazon S3**: Store state files in Amazon S3 buckets, which can be secured and versioned
- **Azure Blob Storage**: Azure users can leverage Azure Blob Storage for storing remote state files
- **HashiCorp Terraform Cloud**: A Terraform Cloud workspace serves as a centralized remote backend that provides version control, collaboration, and additional features

Let's look at an example of configuring Terraform for Azure Blob Storage backend. First before configuring this, you will need to have an Azure Storage Account. Then in your Terraform configuration file (typically `main.tf`) you can configure the backend to use Blob Storage, as such:

```hsl
# main.tf

# Configure Terraform to use Azure Blob Storage as a remote backend
terraform {
  backend "azurerm" {
    resource_group_name   = "your-resource-group"         # Replace with your resource group name
    storage_account_name  = "your-storage-account"        # Replace with your storage account name
    container_name        = "your-container-name"          # Replace with your container name
    key                   = "terraform.tfstate"            # Specify the state file name
  }
}
```

In this example, you would need to replace the placeholders with the actual values:

- `"your-resource-group"`: The name of the Azure resource group containing the storage account
- `"your-storage-account"`: The name of your Azure Storage Account
- `"your-container-name"`: The name of the container within the storage account where the state file will be stored

### Importance of State Locking

> **State locking** is a mechanism used in Terraform to prevent simultaneous writes to the Terraform state file. The Terraform state file contains the current state of the managed infrastructure, including resource IDs, attribute values, and other critical information.

State locking typically involves acquiring a lock on the state file before any modifications are made and releasing the lock once the modifications are complete. This ensures that only one user or process can make changes to the state file at any given time, preventing conflicts and data corruption.

State locking can be achieved through different methods, depending on the chosen state backend:

- **Remote State Backends**: Many remote state backends (e.g., Amazon S3, Azure Storage) provide built-in locking mechanisms. Terraform interacts with the remote backend to acquire and release locks automatically.

- **Local State Backends**: For local state backends, external locking mechanisms such as file locking are often employed. Terraform users must configure and manage these locking mechanisms themselves.

## Terraform Workspaces

> *Terraform workspaces* enable you to manage multiple instances of the same infrastructure in separate environments, such as development, staging, and production. Each workspace maintains its own state, variables, and configuration settings, providing a clean separation between different environments. This isolation prevents accidental configuration changes and simplifies collaboration.

Each workspace maintains its own state file (e.g., `terraform.tfstate.dev` for the `dev` workspace). This ensures that resources created in one workspace don't interfere with resources in another workspace.

To start using workspaces, you can initialize a new workspace with the `terraform workspace new` command. For example, to create a new workspace named `dev`, you can run:

```shell
terraform workspace new dev
```
This command creates a new workspace and automatically switches to it. You can also switch between workspaces using the `terraform workspace select` command. For example, to switch to the `staging` workspace, you can run:

```shell
terraform workspace select staging
```
This command switches to the `staging` workspace and loads its associated state and configuration.

> When you apply changes using `terraform apply`, Terraform will apply the configuration for the currently selected workspace. For instance, if you're in the `staging` workspace, applying changes will only affect the staging environment.

You can list available workspaces using `terraform workspace list` and delete workspaces using `terraform workspace delete`.

```shell
# List available workspaces
terraform workspace list

# Delete a workspace (e.g., "dev")
terraform workspace delete dev
```

### Create Environment-Specific Variables

In your Terraform configuration files, you can define variables that are specific to each workspace. This allows you to customize configurations for different environments. For instance, you can have variables in a `variables.tf` file like this:

```hcl
variable "region" {
  description = "The AWS region for resources."
  type        = string
  default     = "us-east-1"
}

variable "instance_count" {
  description = "The number of instances to launch."
  type        = number
}
```

Then, you can set workspace-specific values for these variables using environment-specific variable files (e.g., `dev.tfvars`, `staging.tfvars`, and `prod.tfvars`) or directly using the `-var` flag when running Terraform commands.

### Benefits of Environment Isolation with Workspaces

1. **Clean Separation**: Workspaces allow you to maintain a clean separation between different environments, reducing the risk of configuration conflicts or unintended changes

2. **Efficient Collaboration**: Teams can work simultaneously on different environments without interfering with each other's configurations

3. **Easy Environment Replication**: You can easily replicate environments (e.g., clone a staging environment to create a new testing environment)

4. **State Management**: Each workspace maintains its own state, making it easier to manage state files and resources

5. **Variable Customization**: Customize variables for each environment, ensuring flexibility while maintaining code reusability

## Handling Dependencies and Relationships

### Resource Dependencies

Resource dependencies represent the order in which Terraform creates, updates, and deletes resources. We have briefly seen in the previous lesson that you can explicitly define dependencies between resources using the `depends_on` attribute. This ensures that certain resources are created or updated before others.

Let's look at this in more detail:

```hcl
resource "aws_instance" "web" {
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"
}

resource "aws_security_group" "web" {
  name_prefix = "web-"
}

# Define a dependency relationship
resource "aws_network_interface_sg_attachment" "example" {
  security_group_id    = aws_security_group.web.id
  network_interface_id = aws_instance.web.network_interface_ids[0]

  # Explicitly specify the dependency
  depends_on = [aws_instance.web]
}
```

In the example above, the `aws_network_interface_sg_attachment` resource depends on the `aws_instance.web` resource using the `depends_on` attribute. This means that Terraform will ensure the `aws_instance.web` resource is created before attempting to create the `aws_network_interface_sg_attachment` resource.

### Understanding Resource Lifecycle

Terraform resources have three main lifecycle stages: create, update, and delete.

- **Create**: When a resource is created, Terraform provisions the resource in your infrastructure. This typically happens when running `terraform apply`.

- **Update**: During an update, Terraform detects changes to the resource's configuration and attempts to update the resource to match the desired state defined in your configuration. Terraform can perform in-place updates or replace the resource entirely, depending on the changes.

- **Delete**: When a resource is deleted, Terraform destroys the resource in your infrastructure. This typically happens when running `terraform destroy`.

### Managing Resource Drift

> *Resource drift* occurs when the actual state of a resource in your infrastructure differs from the desired state defined in your Terraform configuration. To detect and manage drift, Terraform provides the `terraform refresh` command.

The `terraform refresh` command queries the current state of your resources and updates the Terraform state file without making any changes to your infrastructure. It's a useful tool for identifying differences between your configuration and the real-world state.

## Advanced Provisioning and Configuration

In Terraform, you can go beyond basic resource provisioning by leveraging advanced techniques to customize and manage your infrastructure efficiently. This section explores several advanced concepts and practices, including customizing user data for instances, dynamic block definitions, and using the `for_each` and `count` meta-arguments. 

### Customizing User Data for Instances

In cloud computing, *user data* refers to a feature that allows you to customize the initialization of virtual instances by providing scripts or data that are executed or used when the instance is launched. These scripts are typically used to perform various configuration tasks, install software, and automate setup processes on the virtual machine.

```hcl
resource "aws_instance" "example" {
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"

  user_data = <<-EOF
              #!/bin/bash
              echo "Hello, Terraform!"
              EOF
}
```

Here's a breakdown of the example above:

- `resource "aws_instance" "example"`: This defines an AWS EC2 instance resource named `"example"`. It specifies the configuration for an Amazon Machine Image (AMI) and the instance type.

- `user_data`: This is where user data is specified. It's a multiline string that contains a Bash script. In this example, the Bash script is very simple and just echoes the message `"Hello, Terraform!"`.

When you launch an EC2 instance with this Terraform configuration (or an Azure Virtual Machine), the user data script specified in the `user_data` attribute will be executed during the instance initialization. This script can be used for a wide range of tasks, such as:

- Installing and configuring software
- Running updates and patches on the instance
- Downloading and setting up files and configurations
- Executing any other custom initialization tasks

### Dynamic Block Definitions

*Dynamic block definitions* in Terraform allow you to create and manage resource configurations dynamically. This is particularly useful when you have a variable number of similar resources to provision.

```hcl
variable "subnets" {
  type    = list(string)
  default = ["subnet-1", "subnet-2"]
}

resource "aws_security_group_rule" "example" {
  type        = "ingress"
  from_port   = 80
  to_port     = 80
  protocol    = "tcp"

  dynamic "cidr_blocks" {
    for_each = var.subnets
    content {
      cidr_block = "10.0.0.0/24"
    }
  }

  security_group_id = aws_security_group.example.id
}
```

In this Terraform example, dynamic block definitions are being used to create and manage AWS security group rules for a variable number of subnets. This is useful when you have multiple subnets, and you want to create security group rules that allow incoming traffic on port 80 (HTTP) to each of these subnets.

Let's break down the code:

- `variable "subnets"`: This defines a variable called `"subnets"` of type `list(string)`. It is used to hold a list of subnet IDs, and it has a default value of `["subnet-1", "subnet-2"]`. You can provide different subnet IDs as needed.

- `resource "aws_security_group_rule" "example"`: This defines an AWS security group rule resource called `"example"`. This resource is used to manage incoming traffic rules. Within this resource block there is a dynamic block:

  - `dynamic "cidr_blocks"`: This creates a dynamic block with the label `"cidr_blocks"`. It will be dynamically generated based on the elements of the `var.subnets` list.

  - `for_each = var.subnets`: The `for_each` argument specifies that a separate instance of the `"cidr_blocks"` block should be created for each element in the `var.subnets` list. In this case, it means an instance for each subnet ID.

  - `content`: Within the `"cidr_blocks"` block, there's a `cidr_block` attribute, which specifies the CIDR block to allow traffic from. In this example, it's set to `"10.0.0.0/24"`, which is a hard coded value for the CIDR block.

With this configuration, Terraform will dynamically create a security group rule for each subnet specified in the `var.subnets` list. The rules will allow incoming traffic on port 80 (HTTP) from the CIDR block `"10.0.0.0/24"` to each of the specified subnets. This is a powerful way to create and manage resource configurations dynamically based on variable inputs, and it's particularly useful when you have a variable number of similar resources to provision.

### Using `for_each` and `count` Meta-Arguments

The `for_each` and `count` meta-arguments provide powerful ways to create multiple instances of a resource or customize resource configurations based on input data.

Using `for_each`:

```hcl
variable "instance_names" {
  type    = map(string)
  default = {
    web     = "Web Server"
    db      = "Database Server"
    app     = "Application Server"
  }
}

resource "aws_instance" "example" {
  for_each = var.instance_names

  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"

  tags = {
    Name = each.key
    Role = each.value
  }
}
```

In this example, the `for_each` meta-argument is applied to the `aws_instance` resource. It instructs Terraform to create instances for each key-value pair in the `var.instance_name`s map. For example, it will create an instance with the key `"web"` and the corresponding value `"Web Server"`. 

With this configuration, when you apply it using Terraform, it will create three AWS EC2 instances, each with a different role and description as specified in the `var.instance_names` map. 

Using `count`:

```hcl
variable "instance_count" {
  type    = number
  default = 3
}

resource "aws_instance" "example" {
  count         = var.instance_count
  ami           = "ami-0c55b159cbfafe1f0"
  instance_type = "t2.micro"
}
```

The `count` meta-argument is applied to the `aws_instance` resource. It specifies how many instances should be created. In this case, it uses the value of the `"instance_count"` variable, which is set to 3 by default.


## Key Takeaways

- Terraform maintains a state file that tracks the current state of managed resources. Local state files reside on your machine and can lead to collaboration challenges and state drift. Storing state remotely, in tools like AWS S3 or Terraform Cloud, improves collaboration and consistency.
- Workspaces enable environment isolation within a single configuration, allowing you to manage multiple versions of your infrastructure. Use workspaces for managing development, staging, and production environments separately.
- Workspaces allow you to set workspace-specific variable values, enhancing configuration flexibility
- Use the `depends_on` attribute to define explicit dependencies between resources, ensuring they are provisioned in the correct order
- Customize cloud instances with user data scripts for tasks like software installation and configuration
- Utilize `for_each` and `count` meta-arguments to create and customize multiple instances of a resource